In [1]:
import pandas as pd
import numpy as np

In [2]:
path_tabla = '/Users/santiagotoso/GoogleDrive/Master/Python/MiBus/CONF_LINEA 190708 Tabla homologación Remix.xlsx'
path_gtfs = '/Users/santiagotoso/GoogleDrive/Master/Python/MiBus/GTFS_20190624/routes.txt'

tabla = pd.read_excel(path_tabla)
routes = pd.read_csv(path_gtfs)

## Routes.txt

In [3]:
tabla.head()

,TRAYECTO,SENTIDO,SECTOR,ORIGEN,DESTINO/PUNTO INFLEXIÓN,REMIX
0,T040-I,I,Alcalde Diaz,Entrada Alcalde Diaz,Albrook,T040 Alcalde Diaz Albrook
1,T040-R,R,Alcalde Diaz,Albrook,Meconsa-I,T040 Alcalde Diaz Albrook
2,K010,Circular,Altos de Los Lagos,Etapa 1-R,Calle 13 Colon-R,K010 Altos de Los Lagos Calle 13
3,T582-I,I,Los Pueblos,Los Pueblos-R,Zona Paga 5 de Mayo,T582 San Antonio ZP 5 de Mayo
4,T582-R,R,Los Pueblos,Zona Paga 5 de Mayo,Los Pueblos-I,T582 San Antonio ZP 5 de Mayo


In [5]:
routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
0,C640-I,MIBUS,C640,Panama Viejo-Av Balboa-Albrook,3,87838a,ffffff
1,C640-R,MIBUS,C640,Cinta Costera-Panama Viejo-El Balboa,3,87838a,ffffff
2,C641-I,MIBUS,C641,Panama Viejo-Via Porras-Albrook-Directo,3,87838a,ffffff
3,C641-R,MIBUS,C641,Directo-Cincuentenario-Panama Viejo-El Balboa,3,87838a,ffffff
4,C642-I,MIBUS,C642,Panama Viejo-Av Balboa-M Mariscos,3,87838a,ffffff


In [9]:
# Keep only the columns we care about to make it simpler
tabla1 = tabla.loc[:, ['TRAYECTO', 'REMIX']]
tabla1.head(1)

,TRAYECTO,REMIX
0,T040-I,T040 Alcalde Diaz Albrook


In [11]:
# Join the two dataframes
routes1 = pd.merge(routes, tabla1, left_on='route_id', right_on='TRAYECTO', how='left')
routes1.head(2)

,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color,TRAYECTO,REMIX
0,C640-I,MIBUS,C640,Panama Viejo-Av Balboa-Albrook,3,87838a,ffffff,C640-I,C640 Panamá Viejo Albrook
1,C640-R,MIBUS,C640,Cinta Costera-Panama Viejo-El Balboa,3,87838a,ffffff,C640-R,C640 Panamá Viejo Albrook


In [13]:
# Remove the route_long_name
# Take "REMIX" as route_short_name
# Drop the columns we don't need
routes1['route_long_name'] = ''
routes1['route_short_name'] = routes1['REMIX']
routes1.drop(['TRAYECTO', 'REMIX'],axis=1, inplace=True)
routes1.head()

,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
0,C640-I,MIBUS,C640 Panamá Viejo Albrook,,3,87838a,ffffff
1,C640-R,MIBUS,C640 Panamá Viejo Albrook,,3,87838a,ffffff
2,C641-I,MIBUS,C641 Panamá Viejo Albrook,,3,87838a,ffffff
3,C641-R,MIBUS,C641 Panamá Viejo Albrook,,3,87838a,ffffff
4,C642-I,MIBUS,C642 Panamá Viejo Mercado de Mariscos,,3,87838a,ffffff


In [21]:
# Droping duplicate values 
routes1.drop_duplicates(subset = "route_short_name", 
                     keep = 'first', inplace = True)
routes1.head()

,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
0,C640-I,MIBUS,C640 Panamá Viejo Albrook,,3,87838a,ffffff
2,C641-I,MIBUS,C641 Panamá Viejo Albrook,,3,87838a,ffffff
4,C642-I,MIBUS,C642 Panamá Viejo Mercado de Mariscos,,3,87838a,ffffff
6,C800,MIBUS,C800 Parque Summit Albrook,,3,87838a,ffffff
7,C810-I,MIBUS,C810 Miraflores Albrook,,3,87838a,ffffff


In [28]:
# We'll make sure all the route_ids finish with -I so we can easily replace them in trips.txt

routes1['route_id'] = routes1['route_id'].str.replace('-R', '-I')

## Trips.txt

In [22]:
trips = pd.read_csv( '/Users/santiagotoso/GoogleDrive/Master/Python/MiBus/GTFS_20190624/trips.txt')
trips.head()

,route_id,service_id,trip_id,trip_short_name,direction_id,shape_id,wheelchair_accessible
0,C970-R,C970_1,SC0001H-PT1-43642-005*C970-R-VS0001H-01-01,C970 Mercapanama,0,C970_R,1
1,C970-I,C970_1,SC0001H-PT1-43642-006*C970-I-VS0001H-01-02,C970 Albrook,1,C970_I,1
2,C970-R,C970_1,SC0001H-PT1-43642-007*C970-R-VS0001H-01-03,C970 Mercapanama,0,C970_R,1
3,C970-I,C970_1,SC0001H-PT1-43642-008*C970-I-VS0001H-01-04,C970 Albrook,1,C970_I,1
4,C970-R,C970_1,SC0001H-PT1-43642-009*C970-R-VS0001H-01-05,C970 Mercapanama,0,C970_R,1


In [24]:
# Replace the route_ids with -I to match the route_ids we have in routes.txt

trips['route_id'] = trips['route_id'].str.replace('-R', '-I')
trips.head()

,route_id,service_id,trip_id,trip_short_name,direction_id,shape_id,wheelchair_accessible
0,C970-I,C970_1,SC0001H-PT1-43642-005*C970-R-VS0001H-01-01,C970 Mercapanama,0,C970_R,1
1,C970-I,C970_1,SC0001H-PT1-43642-006*C970-I-VS0001H-01-02,C970 Albrook,1,C970_I,1
2,C970-I,C970_1,SC0001H-PT1-43642-007*C970-R-VS0001H-01-03,C970 Mercapanama,0,C970_R,1
3,C970-I,C970_1,SC0001H-PT1-43642-008*C970-I-VS0001H-01-04,C970 Albrook,1,C970_I,1
4,C970-I,C970_1,SC0001H-PT1-43642-009*C970-R-VS0001H-01-05,C970 Mercapanama,0,C970_R,1


## Saving the files

In [29]:
routes1.to_csv(path_gtfs, index=False)
trips.to_csv('/Users/santiagotoso/GoogleDrive/Master/Python/MiBus/GTFS_20190624/trips.txt', index=False)